In [31]:
import json
import numpy as np
import re
import os
import nltk
import pandas as pd
from keras.utils import np_utils


from keras.layers import Dense ,LSTM,concatenate,Input,Flatten
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Activation, merge, Flatten, Reshape
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers

# Problem Forumlation

### Model is provided two inputs question_vector and document_vector and asked to predict the start and end index of answer in the document

# Load word embeddings

In [32]:
embedding_path = "./GloVe/glove.840B.300d.txt"
embedding_path = "/Users/rajatjain/Documents/new_webapp/react-app-kubernetes/backend/src/app/data/glove/glove.6B.100d.txt"

In [33]:
train = pd.read_json('./data/train-v1.1.json')

In [63]:
documents = []
questions = []
answers = []
titles = []
answer_start_indexs = []
answer_end_indexs = []
def get_attributes(item):
    data = item['data']
    title = data['title']
    for paragraph in data['paragraphs']:
        for qas in paragraph['qas']:
            answer = qas['answers'][0]['text']
            answer_start_index = qas['answers'][0]['answer_start']
            answer_end_index = answer_start_index + len(answer.split(' ')) - 1
            answers.append(qas['answers'][0]['text'])
            questions.append(qas['question'])
            documents.append(paragraph['context'])
            answer_start_indexs.append(answer_start_index)
            answer_end_indexs.append(answer_end_index)
            
            titles.append(title)
            
def build_dataframe(train):
    train.apply(get_attributes, axis = 1)
    train_df = pd.DataFrame({
    'documents':documents,
    'questions': questions,
    'answers': answers,
    'titles': titles,
     'answer_end_indexs': answer_end_indexs,
    'answer_start_indexs': answer_start_indexs
})
    return train_df
    
train_df = build_dataframe(train)
train_df = train_df.head(5000)

In [64]:
train_df.head()

,documents,questions,answers,titles,answer_end_indexs,answer_start_indexs
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,University_of_Notre_Dame,517,515
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,University_of_Notre_Dame,192,188
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,University_of_Notre_Dame,281,279
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,University_of_Notre_Dame,387,381
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,University_of_Notre_Dame,98,92


In [43]:
def get_max_length(sentences):
    max_length = 0
    for sentence in sentences:
        length_of_sentence = len(sentence)
        if length_of_sentence > max_length:
            max_length = length_of_sentence
    return max_length

# Extract Entities

In [67]:
train_df = train_df.head(2000)
documents = list(train_df["documents"])
questions = list(train_df["questions"])
answer_start_indexs = train_df["answer_start_indexs"].values
answer_end_indexs = train_df["answer_end_indexs"].values
sentences = documents + questions

In [68]:
questions = train_df['questions'].values
answers = train_df['answers'].values
documents = train_df['documents'].values


# Vectorize Question, Answer and Context

In [70]:
vectorized_data = []
def vectorize(item):
    tokenizer = Tokenizer(
    num_words = 20000,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~'   
)
        
    documents = list(item["documents"])
    questions = list(item["questions"])
    answers = list(item['answers'])
    start_index = list(item['answer_start_indexs'])
    end_index = list(item['answer_end_indexs'])
    sentences = documents + questions
    
    tokenizer.fit_on_texts(sentences)
    questions_tokenized = tokenizer.texts_to_sequences(questions)
    answers_tokenized = tokenizer.texts_to_sequences(answers)
    documents_tokenized = tokenizer.texts_to_sequences(documents)
    
    questions_padded = pad_sequences(questions_tokenized, maxlen = 80, padding = 'post')
    answers_padded = pad_sequences(answers_tokenized, maxlen = 1405, padding = 'post')
    documents_padded = pad_sequences(documents_tokenized, maxlen = 1405, padding = 'post')
    for i in range(0, len(documents)):
        vectorized_data.append([questions_padded[i], answers_padded[i], documents_padded[i], start_index[i], end_index[i] ])
    
train_df.groupby('documents').apply(vectorize)
vectorized_data = pd.DataFrame(vectorized_data)
vectorized_data.rename(columns = {0: 'question_vector', 1: 'answers_vector', 2: 'documents_vector', 3: 'answer_start_indexs', 4: 'answer_end_indexs' },inplace = True)



In [71]:
vectorized_data.head()

,question_vector,answers_vector,documents_vector,answer_start_indexs,answer_end_indexs
0,"[89, 93, 94, 2, 95, 1, 4, 18, 19, 88, 13, 16, ...","[31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4, 18, 19, 9, 27, 8, 28, 29, 2, 30, 1, 31, 32...",56,57
1,"[8, 90, 97, 98, 2, 5, 21, 7, 88, 13, 16, 99, 1...","[34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4, 18, 19, 9, 27, 8, 28, 29, 2, 30, 1, 31, 32...",73,73
2,"[22, 89, 101, 9, 2, 6, 92, 13, 16, 102, 0, 0, ...","[38, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4, 18, 19, 9, 27, 8, 28, 29, 2, 30, 1, 31, 32...",157,158
3,"[103, 104, 8, 23, 24, 17, 2, 6, 92, 88, 13, 10...","[51, 23, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 18, 19, 9, 27, 8, 28, 29, 2, 30, 1, 31, 32...",284,286
4,"[90, 17, 2, 25, 26, 1, 13, 106, 6, 91, 0, 0, 0...","[71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4, 18, 19, 9, 27, 8, 28, 29, 2, 30, 1, 31, 32...",535,536


# Model Architecture

In [72]:
question_input = Input(shape=(80,), dtype='int32', name='question_input')
context_input =  Input(shape=(1405,), dtype='int32', name='context_input')

questionEmbd = Embedding(output_dim=100, input_dim=20000,
                         mask_zero=False, 
                         input_length=80, trainable=False)(question_input)


contextEmb = Embedding(output_dim=100, input_dim=20000,
                         mask_zero=False,
                         input_length=1405, trainable=False)(context_input)


In [73]:
Q = Bidirectional(LSTM(80, return_sequences=True))(questionEmbd)
D = Bidirectional(LSTM(40, return_sequences=True))(contextEmb)
Q_flatten = Flatten()(Q)
D_flatten = Flatten()(D)
merged = concatenate([D_flatten, Q_flatten])

In [74]:
output1 = Dense(1,activation='sigmoid')(merged)
l2_merged = concatenate([merged, output1])
output2 = Dense(1,activation='sigmoid')(l2_merged)

model = Model(inputs=[question_input,context_input], output = [output1,output2])
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 1405)         0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1405, 100)    2000000     context_input[0][0]              
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 80, 100)      2000000     question_input[0][0]             
____________________________________________________________________________________________

/Users/rajatjain/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """


# Model Fit

In [77]:
questions_padded = np.array(vectorized_data['question_vector'].values.tolist())
documents_padded = np.array(vectorized_data['documents_vector'].values.tolist())
answer_begin = np.array(vectorized_data['answers_vector'].values.tolist())
answer_start_indexs = np.array(vectorized_data['answer_start_indexs'].values.tolist())
answer_end_indexs = np.array(vectorized_data['answer_end_indexs'].values.tolist())


In [61]:
history = model.fit([questions_padded, documents_padded],[answer_start_indexs,  answer_end_indexs] ,
                    epochs=10,
                    batch_size=300)

Epoch 1/10
2005/2005 [==============================] - 43s 22ms/step - loss: 415085.9400 - dense_5_loss: 206997.1935 - dense_6_loss: 208088.7495 - dense_5_mean_squared_error: 206997.1935 - dense_6_mean_squared_error: 208088.7495
Epoch 2/10
2005/2005 [==============================] - 46s 23ms/step - loss: 415085.7685 - dense_5_loss: 206997.1017 - dense_6_loss: 208088.6606 - dense_5_mean_squared_error: 206997.1017 - dense_6_mean_squared_error: 208088.6606
Epoch 3/10
2005/2005 [==============================] - 47s 23ms/step - loss: 415085.7692 - dense_5_loss: 206997.1049 - dense_6_loss: 208088.6603 - dense_5_mean_squared_error: 206997.1049 - dense_6_mean_squared_error: 208088.6603
Epoch 4/10
2005/2005 [==============================] - 42s 21ms/step - loss: 415085.7801 - dense_5_loss: 206997.1095 - dense_6_loss: 208088.6667 - dense_5_mean_squared_error: 206997.1095 - dense_6_mean_squared_error: 208088.6667
Epoch 5/10
2005/2005 [==============================] - 54s 27ms/step - loss: 41